In [24]:
# import necessary libraries
import pandas as pd
import os
import textstat
from openai import OpenAI
import json
import re
import requests
from dotenv import load_dotenv

In [25]:
# import prompts 
from jh_pfx_prompts import example, icd10_example, baseline_zeroshot_prompt, single_fewshot_prompt, single_fewshot_icd10_labeling_prompt

In [52]:
os.environ['OPENAI_API_KEY'] = ''

In [27]:
# api key
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
CLIENT = OpenAI(api_key = OPENAI_API_KEY)
OPENAI_MODEL = "gpt-4o"

In [28]:
#reading levels
PROFESSIONAL = "Professional"
COLLEGE_GRADUATE = "College Graduate"
COLLEGE = "College"
TENTH_TO_TWELFTH_GRADE = "10th to 12th grade"
EIGTH_TO_NINTH_GRADE = "8th to 9th grade"
SEVENTH_GRADE = "7th grade"
SIXTH_GRADE = "6th grade"
FIFTH_GRADE = "5th grade"
N_A = "N/A"

In [29]:
# https://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests#Flesch_reading_ease
def map_reading_level(flesch_reading_ease):
    if flesch_reading_ease < 10:
        return PROFESSIONAL
    elif 10.0 <= flesch_reading_ease < 30.0:
        return COLLEGE_GRADUATE
    elif 30.0 <= flesch_reading_ease < 50.0:
        return COLLEGE
    elif 50.0 <= flesch_reading_ease < 60.0:
        return TENTH_TO_TWELTH_GRADE
    elif 60.0 <= flesch_reading_ease < 70.0:
        return EIGTH_TO_NINTH_GRADE
    elif 70.0 <= flesch_reading_ease < 80.0:
        return SEVENTH_GRADE
    elif 80.0 <= flesch_reading_ease < 90.0:
        return SIXTH_GRADE
    elif 90.0 <= flesch_reading_ease < 100.0:
        return FIFTH_GRADE 
    else:
        return N_A

In [43]:
# reading ease variables
fifth_grade = 95
sixth_grade = 85
seventh_grade = 75
eigth_and_ninth_grade = 65
tenth_to_twelfth_grade = 55
college = 40
college_graduate = 20
professional = 5

In [44]:
def adjust_difference(diff, threshold):
    """Adjust the readability difference based on the threshold."""
    if diff > threshold:
        return diff - threshold
    return 0

In [30]:
# import fewshot examples
df_fewshot = pd.read_csv('pfx_fewshot_examples_college.csv')

In [31]:
# import evaluation data 
df_eval = pd.read_csv('pfx_evaluation_data.csv')

In [32]:
print(TENTH_TO_TWELFTH_GRADE)

10th to 12th grade


In [33]:
# extract the json from openai
def extract_json(openai_response):
    if openai_response:  # Ensure the response is not None
        try:
            # Extract content from response object
            content = openai_response.message.content
            
            # Search for JSON within the content
            json_match = re.search(r'```json\n(.*?)\n```', content, re.DOTALL)
            if json_match:
                json_str = json_match.group(1)
                return json.loads(json_str)  # Parse JSON string to Python dict
            else:
                print("No JSON found in response content.")
                return None
        except AttributeError as e:
            print(f"Attribute error: {e}. Ensure the input is a valid response object.")
            return None
    else:
        return None

In [34]:
# takes json object and generated icd10 labels for incidental findings 
def label_icd10s(pfx_outputs_json):
    pfx_icd10_fewshot_examples = ""
    for i, row in df_fewshot.iterrows():
        pfx_icd10_fewshot_examples += icd10_example.format(**row)
    pfx_icd10_codes = []
    pfx_icd10_response = None 
    for pfx_output in pfx_outputs_json :
        try:
            prompt = single_fewshot_icd10_labeling_prompt.format(examples = pfx_icd10_fewshot_examples, PFx = pfx_output['PFx'])
        except:
            print("ERROR: %S" % pfx_output)
            return
        pfx_icd10_response = CLIENT.chat.completions.create(
            model = OPENAI_MODEL,
            temperature = 0.0,
            messages = [
                {"role": "system", "content": "You are an ICD10 medical coder for incidental findings. "},
                {"role": "system", "content": prompt}
            ],
            stream = False,
        )
    pfx_icd10_codes.append(pfx_icd10_response.choices[0])
    return list(map(extract_json, pfx_icd10_codes))

In [35]:
pfx_fewshot_examples = ""
for i, row in df_fewshot.iterrows():
    pfx_fewshot_examples += example.format(**row)

pfx_fewshot_outputs = []
for i, row in df_eval.iloc[:1].iterrows():
    prompt = single_fewshot_prompt.format(Examples = pfx_fewshot_examples, Incidental_Finding = row['Incidental_Finding'], Reading_Level = TENTH_TO_TWELFTH_GRADE)
pfx_response = None 
pfx_response = CLIENT.chat.completions.create(
    model = OPENAI_MODEL,
    temperature = 0.0,
    messages = [
        {"role": "system", "content": "You are a md doctor rephrasing and explaining medical terminology to a patient in an understandable manor."},
        {"role": "system", "content": prompt}
    ],
    stream = False,
)

pfx_fewshot_outputs.append(pfx_response.choices[0])

In [36]:
pfx_fewshot_outputs_json = list(map(extract_json, pfx_fewshot_outputs))

In [37]:
pfx_fewshot_outputs_json

[{'finding': 'White matter lesions',
  'ICD10_code': 'R90.89',
  'PFx': "White matter lesions are areas of the brain that appear different on an MRI scan. These changes are often found incidentally and can be associated with aging, high blood pressure, or other conditions. Patients should understand that while these lesions are common, they do not always cause symptoms or require treatment. However, they can sometimes be linked to issues like memory problems or balance difficulties. It's important to maintain a healthy lifestyle, including managing blood pressure and cholesterol, to potentially reduce the risk of further changes. If any symptoms develop, further evaluation may be necessary.",
  'PFx_ICD10_code': 'R90.89'}]

In [38]:
pfx_fewshot_output_df = pd.DataFrame(pfx_fewshot_outputs_json)
pfx_fewshot_outputs_icd10_labels = label_icd10s(pfx_fewshot_outputs_json)

In [40]:
pfx_fewshot_output_df['_0_agent_icd10_codes'] = list(map(lambda x: list(x.values())[0] if x else "", pfx_fewshot_outputs_icd10_labels))
pfx_fewshot_output_df["_0_icd10_matches"]= pfx_fewshot_output_df.ICD10_code == pfx_fewshot_output_df._0_agent_icd10_codes
pfx_fewshot_output_df["_0_pfx_icd10_matches"] = pfx_fewshot_output_df.ICD10_code == pfx_fewshot_output_df["PFx_ICD10_code"] 
pfx_fewshot_output_df["_0_flesch_kincaid"] = pfx_fewshot_output_df['PFx'].apply(textstat.flesch_kincaid_grade)
pfx_fewshot_output_df["_0_flesch"] = pfx_fewshot_output_df['PFx'].apply(textstat.flesch_reading_ease)
pfx_fewshot_output_df["_0_reading_level"] = pfx_fewshot_output_df['_0_flesch'].apply(map_reading_level)
pfx_fewshot_output_df["_0_reading_level_matches"] = pfx_fewshot_output_df['_0_reading_level'] == TENTH_TO_TWELFTH_GRADE

In [41]:
pfx_fewshot_output_df 

,finding,ICD10_code,PFx,PFx_ICD10_code,_0_agent_icd10_codes,_0_icd10_matches,_0_pfx_icd10_matches,_0_flesch_kincaid,_0_flesch,_0_reading_level,_0_reading_level_matches
0,White matter lesions,R90.89,White matter lesions are areas of the brain th...,R90.89,R90.89,True,True,10.6,46.98,College,False


In [49]:
desired_reading_ease = tenth_to_twelfth_grade

# Calculate accuracy score
accuracy_icd10_matches = sum(pfx_fewshot_output_df["_0_icd10_matches"]) / len(pfx_fewshot_output_df.index)
accuracy_pfx_matches = sum(pfx_fewshot_output_df["_0_pfx_icd10_matches"]) / len(pfx_fewshot_output_df.index)

# Adjust weights for overall score
# Calculate readability score 
readability_score = pfx_fewshot_output_df["_0_flesch"]
readability_difference = abs(readability_score - desired_reading_ease)
if desired_reading_ease >= 55:
    # Adjust element-wise if readability_difference is a Series
    readability_difference = readability_difference.apply(lambda x: x - 5 if x > 5 else 0)
elif desired_reading_ease >= 20:
    readability_difference = readability_difference.apply(lambda x: x - 10 if x > 10 else 0)


# Compute the overall score
overall_score = (accuracy_icd10_matches * 0.35) + (accuracy_pfx_matches * 0.35) - (readability_difference * 0.3)

grades_data = []
grades_data.append({
    "accuracy_agent_icd10": float(accuracy_icd10_matches),  # Assuming accuracy is already a scalar
    "accuracy_pfx_icd10": float(accuracy_pfx_matches),
    "readability_difference": float(readability_difference.iloc[0]),  # Use .iloc[0] to extract scalar
    "overall_score": float(overall_score.iloc[0])  # Use .iloc[0] to extract scalar
}) 
grades = pd.DataFrame(grades_data)


In [50]:
print(pfx_fewshot_output_df["_0_reading_level"])
print(type(pfx_fewshot_output_df["_0_reading_level"]))


0    College
Name: _0_reading_level, dtype: object
<class 'pandas.core.series.Series'>


In [51]:
grades

,accuracy_agent_icd10,accuracy_pfx_icd10,readability_difference,overall_score
0,1.0,1.0,3.02,-0.206
